In [ ]:
"""
This is for replicating what was done for the baseline open source models but with more recent models including llama, mistral, gemma, and qwen
"""

In [ ]:
import pandas as pd
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig
)
import torch
from scipy.stats import ttest_rel
import gc
from tqdm import tqdm
import torch.nn.functional as F


In [ ]:
def clean_up():
    """Clean up GPU memory"""
    if 'model' in globals():
        del globals()['model']
    if 'tokenizer' in globals():
        del globals()['tokenizer']
    gc.collect()
    torch.cuda.empty_cache()

def score_sentence(sentence, model, tokenizer, device):
    """Score a sentence using causal language model"""
    inputs = tokenizer(sentence, return_tensors='pt')
    input_ids = inputs['input_ids'].to(device)

    with torch.no_grad():
        # Set use_cache=False to potentially avoid issues with certain model architectures
        outputs = model(input_ids, labels=input_ids, use_cache=False)
        loss = outputs.loss
        neg_log_likelihood = loss.item() * input_ids.size(1)

    return -neg_log_likelihood  # higher = more likely

def obtain_stereotype_scores(stereo_df, model, tokenizer, prefix=None):
    """Obtain stereotype scores for all pairs in dataframe"""
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    stereo_scores = []
    antistereo_scores = []
    preferences = []

    for _, row in tqdm(stereo_df.iterrows(), total=len(stereo_df), desc="Scoring pairs"):
        identity = row['Identity Term']
        stereo = row['Canonical Term Attributes']
        anti = row['Anti-Stereotype Terms']

        # Add prefix if specified
        identity_augmented = f"{prefix} {identity}" if prefix else identity

        stereo_sentence = f"{identity_augmented.capitalize()} are {stereo}."
        antistereo_sentence = f"{identity_augmented.capitalize()} are {anti}."

        s_score = score_sentence(stereo_sentence, model, tokenizer, device)
        a_score = score_sentence(antistereo_sentence, model, tokenizer, device)

        stereo_scores.append(s_score)
        antistereo_scores.append(a_score)
        preferences.append("stereo" if s_score > a_score else "anti")

    return stereo_scores, antistereo_scores, preferences

def get_evaluation(stereo_df, model_name):
    """Compute and display evaluation metrics"""
    print(f'\n{"="*60}')
    print(f'EVALUATION RESULTS FOR: {model_name}')
    print(f'{"="*60}\n')

    rows = []

    # Overall Evaluation
    overall_bpr = (stereo_df[f'{model_name}_preferred'] == 'stereo').mean()
    overall_diff = (stereo_df[f'{model_name}_stereotype_score'] -
                    stereo_df[f'{model_name}_antistereotype_score']).mean()
    t_stat, p_value = ttest_rel(
        stereo_df[f'{model_name}_stereotype_score'],
        stereo_df[f'{model_name}_antistereotype_score']
    )
    rows.append({
        "Stereotype Type": "Overall",
        "BPR": round(overall_bpr, 2),
        "Mean Diff (S - AS)": round(overall_diff, 3),
        "T-stat": round(t_stat, 3),
        "P-value": round(p_value, 4)
    })

    # Evaluation per Stereotype Type
    for stype in sorted(stereo_df['Stereotype Type'].unique()):
        subset = stereo_df[stereo_df['Stereotype Type'] == stype]
        bpr = (subset[f'{model_name}_preferred'] == 'stereo').mean()
        mean_diff = (subset[f'{model_name}_stereotype_score'] -
                     subset[f'{model_name}_antistereotype_score']).mean()
        t_stat, p_value = ttest_rel(
            subset[f'{model_name}_stereotype_score'],
            subset[f'{model_name}_antistereotype_score']
        )
        rows.append({
            "Stereotype Type": stype,
            "BPR": round(bpr, 2),
            "Mean Diff (S - AS)": round(mean_diff, 3),
            "T-stat": round(t_stat, 3),
            "P-value": round(p_value, 4)
        })

    result_df = pd.DataFrame(rows)
    print(result_df.to_string(index=False))
    print()

## The dataset

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

stereo_df = pd.read_csv("stereo_antistereo_pairs.csv")
stereo_df = stereo_df.drop(columns=['Attributes'], errors='ignore')
print(f"\nLoaded {len(stereo_df)} stereotype pairs")
print(f"Stereotype types: {stereo_df['Stereotype Type'].unique()}\n")

Using device: cuda

Loaded 123 stereotype pairs
Stereotype types: ['women' 'men' 'religion' 'age' 'profession' 'ethnicity' 'region' 'other']



## 4-BIT quantization

In [ ]:
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)


### 1: LLAMA 3.2 3B

In [ ]:
print("EVALUATING: Llama 3.2 3B (September 2024)")

clean_up()

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-3B")
model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-3.2-3B",
    torch_dtype=torch.float16,
    device_map="auto"
)
model.eval()

# Standard evaluation
stereo_scores, antistereo_scores, preferences = obtain_stereotype_scores(
    stereo_df, model, tokenizer
)

model_name = 'llama32_3b'
stereo_df[f'{model_name}_stereotype_score'] = stereo_scores
stereo_df[f'{model_name}_antistereotype_score'] = antistereo_scores
stereo_df[f'{model_name}_preferred'] = preferences
get_evaluation(stereo_df, model_name)

# African prefix evaluation
stereo_scores_afr, antistereo_scores_afr, preferences_afr = obtain_stereotype_scores(
    stereo_df, model, tokenizer, prefix="African"
)

model_name = 'llama32_3b_afr'
stereo_df[f'{model_name}_stereotype_score'] = stereo_scores_afr
stereo_df[f'{model_name}_antistereotype_score'] = antistereo_scores_afr
stereo_df[f'{model_name}_preferred'] = preferences_afr
get_evaluation(stereo_df, model_name)

EVALUATING: Llama 3.2 3B (September 2024)


tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/844 [00:00<?, ?B/s]

...

Scoring pairs: 100%|██████████| 123/123 [00:12<00:00,  9.47it/s]
/usr/local/lib/python3.12/dist-packages/scipy/stats/_stats_py.py:1243: RuntimeWarning: divide by zero encountered in divide
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero
/usr/local/lib/python3.12/dist-packages/scipy/stats/_stats_py.py:1243: RuntimeWarning: invalid value encountered in scalar multiply
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero



EVALUATION RESULTS FOR: llama32_3b

Stereotype Type  BPR  Mean Diff (S - AS)  T-stat  P-value
        Overall 0.78               2.481   7.326   0.0000
            age 0.95               3.351   6.255   0.0000
      ethnicity 0.88               1.366   0.913   0.3916
            men 0.69               1.838   2.437   0.0214
          other 1.00               4.244     NaN      NaN
     profession 0.80               3.677   4.249   0.0008
         region 0.50              -0.335  -0.220   0.8623
       religion 0.73               2.254   2.089   0.0554
          women 0.74               2.410   3.237   0.0029



Scoring pairs: 100%|██████████| 123/123 [00:11<00:00, 10.61it/s]


EVALUATION RESULTS FOR: llama32_3b_afr

Stereotype Type  BPR  Mean Diff (S - AS)  T-stat  P-value
        Overall 0.74               1.901   5.699   0.0000
            age 0.86               2.716   3.965   0.0007
      ethnicity 0.75               0.883   0.692   0.5110
            men 0.69               1.731   2.902   0.0072
          other 0.00              -1.055     NaN      NaN
     profession 0.80               2.432   3.058   0.0085
         region 0.00              -2.337  -2.255   0.2657
       religion 0.73               1.506   1.358   0.1960
          women 0.74               2.048   2.540   0.0165




/usr/local/lib/python3.12/dist-packages/scipy/stats/_stats_py.py:1243: RuntimeWarning: divide by zero encountered in divide
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero
/usr/local/lib/python3.12/dist-packages/scipy/stats/_stats_py.py:1243: RuntimeWarning: invalid value encountered in scalar multiply
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero


### 2: PHI-3 MINI 3.8B

In [ ]:
print("EVALUATING: Phi-3 Mini 3.8B (June 2024)")

clean_up()

tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)
model.eval()

# Standard evaluation
stereo_scores, antistereo_scores, preferences = obtain_stereotype_scores(
    stereo_df, model, tokenizer
)

model_name = 'phi3_mini'
stereo_df[f'{model_name}_stereotype_score'] = stereo_scores
stereo_df[f'{model_name}_antistereotype_score'] = antistereo_scores
stereo_df[f'{model_name}_preferred'] = preferences
get_evaluation(stereo_df, model_name)

# African prefix evaluation
stereo_scores_afr, antistereo_scores_afr, preferences_afr = obtain_stereotype_scores(
    stereo_df, model, tokenizer, prefix="African"
)

model_name = 'phi3_mini_afr'
stereo_df[f'{model_name}_stereotype_score'] = stereo_scores_afr
stereo_df[f'{model_name}_antistereotype_score'] = antistereo_scores_afr
stereo_df[f'{model_name}_preferred'] = preferences_afr
get_evaluation(stereo_df, model_name)

EVALUATING: Phi-3 Mini 3.8B (June 2024)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Scoring pairs: 100%|██████████| 123/123 [00:11<00:00, 10.47it/s]
/usr/local/lib/python3.12/dist-packages/scipy/stats/_stats_py.py:1243: RuntimeWarning: divide by zero encountered in divide
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero
/usr/local/lib/python3.12/dist-packages/scipy/stats/_stats_py.py:1243: RuntimeWarning: invalid value encountered in scalar multiply
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero



EVALUATION RESULTS FOR: phi3_mini

Stereotype Type  BPR  Mean Diff (S - AS)  T-stat  P-value
        Overall 0.70               1.687   4.781   0.0000
            age 0.86               2.183   3.951   0.0007
      ethnicity 0.62              -1.766  -1.019   0.3421
            men 0.66               1.922   2.815   0.0088
          other 1.00               2.362     NaN      NaN
     profession 0.87               2.855   4.593   0.0004
         region 0.50              -2.423  -0.930   0.5229
       religion 0.60               0.829   0.813   0.4297
          women 0.61               2.101   2.430   0.0213



Scoring pairs: 100%|██████████| 123/123 [00:11<00:00, 10.59it/s]


EVALUATION RESULTS FOR: phi3_mini_afr

Stereotype Type  BPR  Mean Diff (S - AS)  T-stat  P-value
        Overall 0.59               0.897   2.716   0.0076
            age 0.68               1.871   2.945   0.0077
      ethnicity 0.62              -1.205  -0.703   0.5048
            men 0.62               0.748   1.174   0.2502
          other 0.00              -1.517     NaN      NaN
     profession 0.60               0.316   0.454   0.6570
         region 0.00              -2.685  -1.277   0.4228
       religion 0.47               0.801   0.922   0.3719
          women 0.58               1.526   1.927   0.0635




/usr/local/lib/python3.12/dist-packages/scipy/stats/_stats_py.py:1243: RuntimeWarning: divide by zero encountered in divide
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero
/usr/local/lib/python3.12/dist-packages/scipy/stats/_stats_py.py:1243: RuntimeWarning: invalid value encountered in scalar multiply
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero


### 3: MISTRAL 7B

In [ ]:
print("EVALUATING: Mistral 7B v0.1 (2023)")

clean_up()

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")
model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-v0.1",
    quantization_config=quantization_config,
    device_map="auto",
    torch_dtype=torch.float16
)
model.eval()

# Standard evaluation
stereo_scores, antistereo_scores, preferences = obtain_stereotype_scores(
    stereo_df, model, tokenizer
)

model_name = 'mistral_7b'
stereo_df[f'{model_name}_stereotype_score'] = stereo_scores
stereo_df[f'{model_name}_antistereotype_score'] = antistereo_scores
stereo_df[f'{model_name}_preferred'] = preferences
get_evaluation(stereo_df, model_name)

# African prefix evaluation
stereo_scores_afr, antistereo_scores_afr, preferences_afr = obtain_stereotype_scores(
    stereo_df, model, tokenizer, prefix="African"
)

model_name = 'mistral_7b_afr'
stereo_df[f'{model_name}_stereotype_score'] = stereo_scores_afr
stereo_df[f'{model_name}_antistereotype_score'] = antistereo_scores_afr
stereo_df[f'{model_name}_preferred'] = preferences_afr
get_evaluation(stereo_df, model_name)

EVALUATING: Mistral 7B v0.1 (2023)


tokenizer_config.json:   0%|          | 0.00/996 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

Scoring pairs: 100%|██████████| 123/123 [00:43<00:00,  2.81it/s]
/usr/local/lib/python3.12/dist-packages/scipy/stats/_stats_py.py:1243: RuntimeWarning: divide by zero encountered in divide
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero
/usr/local/lib/python3.12/dist-packages/scipy/stats/_stats_py.py:1243: RuntimeWarning: invalid value encountered in scalar multiply
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero



EVALUATION RESULTS FOR: mistral_7b

Stereotype Type  BPR  Mean Diff (S - AS)  T-stat  P-value
        Overall 0.75               2.614   8.448   0.0000
            age 0.86               3.590   6.009   0.0000
      ethnicity 0.88               2.598   2.184   0.0652
            men 0.62               1.668   2.460   0.0203
          other 1.00               3.655     NaN      NaN
     profession 0.93               4.140   6.352   0.0000
         region 0.50               0.100   0.856   0.5492
       religion 0.67               1.694   2.445   0.0283
          women 0.71               2.645   3.535   0.0013



Scoring pairs: 100%|██████████| 123/123 [00:43<00:00,  2.83it/s]


EVALUATION RESULTS FOR: mistral_7b_afr

Stereotype Type  BPR  Mean Diff (S - AS)  T-stat  P-value
        Overall 0.72               2.090   6.461   0.0000
            age 0.73               2.311   3.006   0.0067
      ethnicity 0.75               1.539   1.247   0.2527
            men 0.69               2.178   4.148   0.0003
          other 0.00              -3.138     NaN      NaN
     profession 0.87               2.879   3.707   0.0023
         region 0.00              -1.302  -1.503   0.3738
       religion 0.73               1.053   1.935   0.0734
          women 0.71               2.500   2.856   0.0077




/usr/local/lib/python3.12/dist-packages/scipy/stats/_stats_py.py:1243: RuntimeWarning: divide by zero encountered in divide
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero
/usr/local/lib/python3.12/dist-packages/scipy/stats/_stats_py.py:1243: RuntimeWarning: invalid value encountered in scalar multiply
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero


### 4: GEMMA 2 2B

In [ ]:
print("EVALUATING: Gemma 2 2B (June 2024)")

clean_up()

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-2b")
model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-2-2b",
    torch_dtype=torch.float16,
    device_map="auto"
)
model.eval()

# Standard evaluation
stereo_scores, antistereo_scores, preferences = obtain_stereotype_scores(
    stereo_df, model, tokenizer
)

model_name = 'gemma2_2b'
stereo_df[f'{model_name}_stereotype_score'] = stereo_scores
stereo_df[f'{model_name}_antistereotype_score'] = antistereo_scores
stereo_df[f'{model_name}_preferred'] = preferences
get_evaluation(stereo_df, model_name)

# African prefix evaluation
stereo_scores_afr, antistereo_scores_afr, preferences_afr = obtain_stereotype_scores(
    stereo_df, model, tokenizer, prefix="African"
)

model_name = 'gemma2_2b_afr'
stereo_df[f'{model_name}_stereotype_score'] = stereo_scores_afr
stereo_df[f'{model_name}_antistereotype_score'] = antistereo_scores_afr
stereo_df[f'{model_name}_preferred'] = preferences_afr
get_evaluation(stereo_df, model_name)

EVALUATING: Gemma 2 2B (June 2024)


tokenizer_config.json:   0%|          | 0.00/46.4k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

...

Scoring pairs: 100%|██████████| 123/123 [00:15<00:00,  7.90it/s]
/usr/local/lib/python3.12/dist-packages/scipy/stats/_stats_py.py:1243: RuntimeWarning: divide by zero encountered in divide
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero
/usr/local/lib/python3.12/dist-packages/scipy/stats/_stats_py.py:1243: RuntimeWarning: invalid value encountered in scalar multiply
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero



EVALUATION RESULTS FOR: gemma2_2b

Stereotype Type  BPR  Mean Diff (S - AS)  T-stat  P-value
        Overall 0.71               3.618   6.068   0.0000
            age 0.86               4.949   5.440   0.0000
      ethnicity 0.62               1.601   0.871   0.4128
            men 0.72               4.750   3.150   0.0039
          other 1.00               2.809     NaN      NaN
     profession 0.87               4.023   2.191   0.0459
         region 0.00              -3.587  -9.659   0.0657
       religion 0.60               3.048   1.999   0.0654
          women 0.61               2.706   2.149   0.0398



Scoring pairs: 100%|██████████| 123/123 [00:15<00:00,  7.88it/s]


EVALUATION RESULTS FOR: gemma2_2b_afr

Stereotype Type  BPR  Mean Diff (S - AS)  T-stat  P-value
        Overall 0.63               2.924   4.658   0.0000
            age 0.68               4.177   2.880   0.0090
      ethnicity 0.62               0.427   0.235   0.8213
            men 0.72               5.188   3.798   0.0007
          other 0.00              -3.742     NaN      NaN
     profession 0.60               1.526   0.928   0.3691
         region 0.00              -5.957 -18.883   0.0337
       religion 0.53               1.347   0.842   0.4140
          women 0.61               2.787   2.104   0.0438




/usr/local/lib/python3.12/dist-packages/scipy/stats/_stats_py.py:1243: RuntimeWarning: divide by zero encountered in divide
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero
/usr/local/lib/python3.12/dist-packages/scipy/stats/_stats_py.py:1243: RuntimeWarning: invalid value encountered in scalar multiply
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero


### 5: QWEN 2.5 7B

In [ ]:
print("EVALUATING: Qwen 2.5 7B (2024)")

clean_up()

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-7B")
model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2.5-7B",
    quantization_config=quantization_config,
    device_map="auto",
    torch_dtype=torch.float16
)
model.eval()

# Standard evaluation
stereo_scores, antistereo_scores, preferences = obtain_stereotype_scores(
    stereo_df, model, tokenizer
)

model_name = 'qwen25_7b'
stereo_df[f'{model_name}_stereotype_score'] = stereo_scores
stereo_df[f'{model_name}_antistereotype_score'] = antistereo_scores
stereo_df[f'{model_name}_preferred'] = preferences
get_evaluation(stereo_df, model_name)

# African prefix evaluation
stereo_scores_afr, antistereo_scores_afr, preferences_afr = obtain_stereotype_scores(
    stereo_df, model, tokenizer, prefix="African"
)

model_name = 'qwen25_7b_afr'
stereo_df[f'{model_name}_stereotype_score'] = stereo_scores_afr
stereo_df[f'{model_name}_antistereotype_score'] = antistereo_scores_afr
stereo_df[f'{model_name}_preferred'] = preferences_afr
get_evaluation(stereo_df, model_name)

EVALUATING: Qwen 2.5 7B (2024)


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

Scoring pairs: 100%|██████████| 123/123 [00:42<00:00,  2.92it/s]
/usr/local/lib/python3.12/dist-packages/scipy/stats/_stats_py.py:1243: RuntimeWarning: divide by zero encountered in divide
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero
/usr/local/lib/python3.12/dist-packages/scipy/stats/_stats_py.py:1243: RuntimeWarning: invalid value encountered in scalar multiply
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero



EVALUATION RESULTS FOR: qwen25_7b

Stereotype Type  BPR  Mean Diff (S - AS)  T-stat  P-value
        Overall 0.71               2.514   6.599   0.0000
            age 0.91               3.925   5.790   0.0000
      ethnicity 0.62               1.920   1.339   0.2223
            men 0.62               2.589   3.221   0.0032
          other 1.00               1.074     NaN      NaN
     profession 1.00               3.552   5.846   0.0000
         region 0.00              -1.789  -2.789   0.2192
       religion 0.53               0.812   0.689   0.5020
          women 0.65               2.243   2.405   0.0225



Scoring pairs: 100%|██████████| 123/123 [00:41<00:00,  2.94it/s]


EVALUATION RESULTS FOR: qwen25_7b_afr

Stereotype Type  BPR  Mean Diff (S - AS)  T-stat  P-value
        Overall 0.62               1.392   3.894   0.0002
            age 0.73               2.348   3.047   0.0061
      ethnicity 0.62               0.751   0.529   0.6135
            men 0.66               1.709   2.815   0.0088
          other 0.00              -4.209     NaN      NaN
     profession 0.67               1.730   2.479   0.0265
         region 0.00              -3.327 -15.096   0.0421
       religion 0.53               0.470   0.500   0.6249
          women 0.58               1.352   1.433   0.1621




/usr/local/lib/python3.12/dist-packages/scipy/stats/_stats_py.py:1243: RuntimeWarning: divide by zero encountered in divide
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero
/usr/local/lib/python3.12/dist-packages/scipy/stats/_stats_py.py:1243: RuntimeWarning: invalid value encountered in scalar multiply
  var *= np.divide(n, n-ddof)  # to avoid error on division by zero


In [ ]:
clean_up()

# Save complete results
output_filename = "afristureo_modern_models_results.csv"
stereo_df.to_csv(output_filename, index=False)
print(f"All evaluations complete!")
print(f"Results saved to: {output_filename}")

# Generate summary table
summary_rows = []
models = [
    'llama32_3b', 'llama32_3b_afr',
    'phi3_mini', 'phi3_mini_afr',
    'mistral_7b', 'mistral_7b_afr',
    'gemma2_2b', 'gemma2_2b_afr',
    'qwen25_7b', 'qwen25_7b_afr'
]

for model_name in models:
    bpr = (stereo_df[f'{model_name}_preferred'] == 'stereo').mean()
    mean_diff = (stereo_df[f'{model_name}_stereotype_score'] -
                 stereo_df[f'{model_name}_antistereotype_score']).mean()
    t_stat, p_value = ttest_rel(
        stereo_df[f'{model_name}_stereotype_score'],
        stereo_df[f'{model_name}_antistereotype_score']
    )
    summary_rows.append({
        "Model": model_name,
        "BPR": round(bpr, 3),
        "Mean Diff": round(mean_diff, 3),
        "T-stat": round(t_stat, 3),
        "P-value": round(p_value, 5)
    })

summary_df = pd.DataFrame(summary_rows)
print("SUMMARY: ALL MODERN MODELS")
print(summary_df.to_string(index=False))
print("\n")


In [ ]:
# Save summary
summary_df.to_csv("afristereo_modern_models_summary.csv", index=False)
print("Summary saved to: afristereo_modern_models_summary.csv\n")


Summary saved to: afristureo_modern_models_summary.csv

